In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [12]:
spray = pd.read_csv('../Kaggle Datasets/spray.csv/spray.csv')
train = pd.read_csv('../Kaggle Datasets/train.csv/train.csv')
weather = pd.read_csv('../Kaggle Datasets/weather.csv/weather.csv')

In [13]:
train.columns = train.columns.str.lower()
weather.columns = weather.columns.str.lower()
spray.columns = spray.columns.str.lower()

In [14]:
spray[spray['time'].isnull()]

,date,time,latitude,longitude
1030,2011-09-07,NaN,41.987092,-87.794286
1031,2011-09-07,NaN,41.987620,-87.794382
1032,2011-09-07,NaN,41.988004,-87.794574
1033,2011-09-07,NaN,41.988292,-87.795486
1034,2011-09-07,NaN,41.988100,-87.796014
...,...,...,...,...
1609,2011-09-07,NaN,41.995876,-87.811615
1610,2011-09-07,NaN,41.995972,-87.810271
1611,2011-09-07,NaN,41.995684,-87.810319
1612,2011-09-07,NaN,41.994724,-87.810415


In [15]:
spray[spray['time'].isnull()]['date'].unique() #all the missing time records are from 2011-09-07 record

array(['2011-09-07'], dtype=object)

In [16]:
spray[spray['date']=='2011-09-07'] #it looks like the time is in chronological order based on index value

,date,time,latitude,longitude
95,2011-09-07,7:52:52 PM,41.981433,-87.787777
96,2011-09-07,7:53:02 PM,41.980998,-87.787778
97,2011-09-07,7:53:12 PM,41.980560,-87.787762
98,2011-09-07,7:53:22 PM,41.980198,-87.787758
99,2011-09-07,7:53:32 PM,41.979752,-87.787765
...,...,...,...,...
2204,2011-09-07,9:40:30 PM,41.990155,-87.817828
2205,2011-09-07,9:40:40 PM,41.989638,-87.817857
2206,2011-09-07,9:40:50 PM,41.989088,-87.817867
2207,2011-09-07,9:41:00 PM,41.988812,-87.817618


In [17]:
print(spray.iloc[1029]['time'])
print()
print(spray.iloc[1614]['time'])

7:44:32 PM

7:46:30 PM


In [18]:
start_datetime = datetime.strptime(spray.iloc[1029]['time'], "%I:%M:%S %p")
end_datetime = datetime.strptime(spray.iloc[1614]['time'], "%I:%M:%S %p")
time_diff = end_datetime - start_datetime
interval = time_diff / 584
time_points = []
for i in range(1, 585):
    new_time = start_datetime + (i * interval)
    time_points.append(new_time.strftime("%I:%M:%S %p"))

time_series = pd.Series(time_points)
spray.loc[1030:1613, 'time'] = time_series.astype(str).values

spray.drop_duplicates(keep='first', inplace=True)

---

In [19]:
weather.duplicated().value_counts()  #no duplicate records

False    2944
dtype: int64

In [20]:
weather['water1'].unique()#all the records for water1 is missing so i will drop this column

array(['M'], dtype=object)

In [21]:
weather.drop(['water1', 'codesum'], axis=1, inplace=True)

In [22]:
weather[weather['depth'] == 'M']['station'].unique() #All the records from station 2 have missing depth value so i will drop this column

array([2], dtype=int64)

In [23]:
weather[weather['snowfall'] == 'M']['station'].unique() #All the records from station 2 have missing snowfall value so i will drop this column

array([2], dtype=int64)

In [24]:
weather[weather['depart']=='M']['station'].unique() #All the records from station 2 have missing depart value so i will drop this column

array([2], dtype=int64)

In [25]:
weather[weather['sunrise'] == '-']['station'].unique() #All the records from station 2 have missing sunrise value so i will drop this column

array([2], dtype=int64)

In [26]:
weather[weather['sunset'] == '-']['station'].unique() #All the records from station 2 have missing sunset value so i will drop this column

array([2], dtype=int64)

In [27]:
station1 = weather[weather['station'] == 1].drop('station', axis=1)
station2 = weather[weather['station'] == 2].drop('station', axis=1)

weather = station1.merge(station2, on=['date'], suffixes=('_stn1', '_stn2'))
weather = weather.drop(['depth_stn2', 'snowfall_stn2', 'depart_stn2', 'sunrise_stn2', 'sunset_stn2'], axis=1)

In [28]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 0 to 1471
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              1472 non-null   object 
 1   tmax_stn1         1472 non-null   int64  
 2   tmin_stn1         1472 non-null   int64  
 3   tavg_stn1         1472 non-null   object 
 4   depart_stn1       1472 non-null   object 
 5   dewpoint_stn1     1472 non-null   int64  
 6   wetbulb_stn1      1472 non-null   object 
 7   heat_stn1         1472 non-null   object 
 8   cool_stn1         1472 non-null   object 
 9   sunrise_stn1      1472 non-null   object 
 10  sunset_stn1       1472 non-null   object 
 11  depth_stn1        1472 non-null   object 
 12  snowfall_stn1     1472 non-null   object 
 13  preciptotal_stn1  1472 non-null   object 
 14  stnpressure_stn1  1472 non-null   object 
 15  sealevel_stn1     1472 non-null   object 
 16  resultspeed_stn1  1472 non-null   float64


In [29]:
weather['date']

0       2007-05-01
1       2007-05-02
2       2007-05-03
3       2007-05-04
4       2007-05-05
           ...    
1467    2014-10-27
1468    2014-10-28
1469    2014-10-29
1470    2014-10-30
1471    2014-10-31
Name: date, Length: 1472, dtype: object

In [30]:
# Assuming you have a DataFrame called 'data' with the 'date' column
weather['date'] = pd.to_datetime(weather['date'])

# Extract year, month, and day into separate columns
weather['year'] = weather['date'].dt.year
weather['month'] = weather['date'].dt.month
weather['day'] = weather['date'].dt.day

In [31]:
weather.select_dtypes(exclude='number').columns

Index(['date', 'tavg_stn1', 'depart_stn1', 'wetbulb_stn1', 'heat_stn1',
       'cool_stn1', 'sunrise_stn1', 'sunset_stn1', 'depth_stn1',
       'snowfall_stn1', 'preciptotal_stn1', 'stnpressure_stn1',
       'sealevel_stn1', 'avgspeed_stn1', 'tavg_stn2', 'wetbulb_stn2',
       'heat_stn2', 'cool_stn2', 'preciptotal_stn2', 'stnpressure_stn2',
       'sealevel_stn2', 'avgspeed_stn2'],
      dtype='object')

In [32]:
col_to_convert = []
col_with_trace = []


for x in weather.select_dtypes(exclude='number').columns:
    if x == 'date':
        continue

    weather[x] = weather[x].str.strip()
    print(x)

    if len(weather[x].unique()) == 1:
        weather.drop(x, axis=1)
        print(f'{x} dropped')
        print('-----')
        continue

    if 'M' in weather[x].unique():
        print(f"There are {len(weather[weather[x] == 'M'])} missing values in {x}")

    if 'T' in weather[x].unique():
        print(f"There are {len(weather[weather[x] == 'T'])} trace values in {x}")
        col_with_trace.append(x)

    if 'M' not in weather[x].unique() and 'T' not in weather[x].unique():
        col_to_convert.append(x)
        print(f"{x} have no missing values or trace values and will be converted to numeric")

    print('-----')

tavg_stn1
tavg_stn1 have no missing values or trace values and will be converted to numeric
-----
depart_stn1
depart_stn1 have no missing values or trace values and will be converted to numeric
-----
wetbulb_stn1
There are 3 missing values in wetbulb_stn1
-----
heat_stn1
heat_stn1 have no missing values or trace values and will be converted to numeric
-----
cool_stn1
cool_stn1 have no missing values or trace values and will be converted to numeric
-----
sunrise_stn1
sunrise_stn1 have no missing values or trace values and will be converted to numeric
-----
sunset_stn1
sunset_stn1 have no missing values or trace values and will be converted to numeric
-----
depth_stn1
depth_stn1 dropped
-----
snowfall_stn1
There are 12 trace values in snowfall_stn1
-----
preciptotal_stn1
There are 163 trace values in preciptotal_stn1
-----
stnpressure_stn1
There are 2 missing values in stnpressure_stn1
-----
sealevel_stn1
There are 5 missing values in sealevel_stn1
-----
avgspeed_stn1
avgspeed_stn1 have 

In [33]:
for col in col_to_convert:
    weather[col] = pd.to_numeric(weather[col], errors='coerce')

In [34]:
#handling missing values

# wetbulb values are almost the same values between each station for each day
weather[weather['wetbulb_stn1'] == 'M'][['wetbulb_stn1','wetbulb_stn2']]

,wetbulb_stn1,wetbulb_stn2
424,M,67
1205,M,63
1206,M,64


In [35]:
mask = weather['wetbulb_stn1'] == 'M'
weather.loc[mask, 'wetbulb_stn1'] = weather.loc[mask, 'wetbulb_stn2']

In [36]:
weather[weather['wetbulb_stn2'] == 'M'][['wetbulb_stn1','wetbulb_stn2']]

,wetbulb_stn1,wetbulb_stn2
1207,68,M


In [37]:
weather.loc[1207, 'wetbulb_stn2'] = weather.loc[1207, 'wetbulb_stn1']

In [38]:
weather[weather['stnpressure_stn1'] == 'M'][['stnpressure_stn1', 'stnpressure_stn2']]

,stnpressure_stn1,stnpressure_stn2
424,M,29.20
1205,M,M


In [39]:
weather[weather['stnpressure_stn2'] == 'M'][['stnpressure_stn1', 'stnpressure_stn2']]

,stnpressure_stn1,stnpressure_stn2
43,29.36,M
1205,M,M


In [40]:
weather.loc[40:50,['stnpressure_stn1', 'stnpressure_stn2']] #stnpressure values are almost the same values between each station for each day

,stnpressure_stn1,stnpressure_stn2
40,29.34,29.39
41,29.41,29.47
42,29.44,29.49
43,29.36,M
44,29.30,29.36
45,29.28,29.34
46,29.26,29.33
47,29.23,29.28
48,29.12,29.19
49,29.26,29.31


In [41]:
#replacing missing values to corresponding stnpressure value or 10 days average
weather.loc[43, 'stnpressure_stn2'] = weather.loc[43, 'stnpressure_stn1']
weather.loc[424, 'stnpressure_stn1'] = weather.loc[424, 'stnpressure_stn2']
weather.loc[1205, 'stnpressure_stn1'] = round(weather.loc[1200:1210].drop(1205)['stnpressure_stn1'].astype('float64').mean(), 2) #10days average as both stn had missing value
weather.loc[1205, 'stnpressure_stn2'] = round(weather.loc[1200:1210].drop(1205)['stnpressure_stn2'].astype('float64').mean(), 2)

In [42]:
weather[weather['sealevel_stn1'] == 'M'][['sealevel_stn1', 'sealevel_stn2']]

,sealevel_stn1,sealevel_stn2
416,M,29.79
497,M,30.09
866,M,30.03
878,M,29.99
1045,M,29.87


In [43]:
weather[weather['sealevel_stn2'] == 'M'][['sealevel_stn1', 'sealevel_stn2']]

,sealevel_stn1,sealevel_stn2
43,30.09,M
872,30.09,M
1033,30.07,M
1371,30.12,M


In [44]:
weather.loc[410:420, ['sealevel_stn1', 'sealevel_stn2']] # sealevels are almost the same values between each station for each day

,sealevel_stn1,sealevel_stn2
410,29.95,29.95
411,30.00,29.99
412,30.01,30.00
413,30.01,29.99
414,29.89,29.87
415,29.88,29.87
416,M,29.79
417,29.70,29.68
418,29.76,29.74
419,29.82,29.80


In [45]:
#missing values in stn1 and stn2 did not overlap so we can use mask method
mask = weather['sealevel_stn1'] == 'M'
weather.loc[mask, 'sealevel_stn1'] = weather.loc[mask, 'sealevel_stn2']
mask = weather['sealevel_stn2'] == 'M'
weather.loc[mask, 'sealevel_stn2'] = weather.loc[mask, 'sealevel_stn1']

In [46]:
weather[weather['preciptotal_stn2']=='M'][['preciptotal_stn1', 'preciptotal_stn2']]

,preciptotal_stn1,preciptotal_stn2
58,0.00,M
59,0.00,M


In [47]:
(weather[weather['preciptotal_stn1']=='0.00']['preciptotal_stn2'] == '0.00').sum()

715

In [48]:
(weather[weather['preciptotal_stn1']=='0.00']['preciptotal_stn2'] != '0.00').sum()

79

In [49]:
#about 90% of weather['preciptotal_stn1']=='0.00' has weather['preciptotal_stn2'] == '0.00' so i shall replace missing value with 0.00
weather.loc[58:59, 'preciptotal_stn2'] = '0.00'

In [50]:
weather[weather['tavg_stn2'] == 'M'][['tavg_stn2', 'tmin_stn2', 'tmax_stn2']]

,tavg_stn2,tmin_stn2,tmax_stn2
3,M,51,78
252,M,46,86
337,M,46,62
818,M,71,100
1033,M,72,84
1105,M,42,71
1250,M,52,91
1255,M,53,84
1262,M,48,76
1289,M,47,80


In [51]:
# Use (tmin + tmax) /2 to fill the null values of tavg
import math

weather['tavg_stn2'] = weather.apply(lambda x: math.ceil((float(x['tmin_stn2']) + float(x['tmax_stn2'])) / 2) if x['tavg_stn2'] == 'M' else x['tavg_stn2'], axis=1)

In [52]:
weather[weather['heat_stn2']=='M'][['heat_stn1', 'heat_stn2', 'tavg_stn1', 'tavg_stn2', 'cool_stn1', 'cool_stn2']]

,heat_stn1,heat_stn2,tavg_stn1,tavg_stn2,cool_stn1,cool_stn2
3,7,M,58,65,0,M
252,0,M,78,66,13,M
337,11,M,54,54,0,M
818,0,M,79,86,14,M
1033,0,M,70,78,5,M
1105,16,M,49,57,0,M
1250,5,M,60,72,0,M
1255,5,M,60,69,0,M
1262,10,M,55,62,0,M
1289,15,M,50,64,0,M


In [53]:
weather.loc[1100:1110,['heat_stn1', 'heat_stn2', 'cool_stn1', 'cool_stn2']] 
#since the values between heat_stn1 and heat_stn2 are similar, i shall use heat_stn1 value for missing heat_stn2. Same for cool_stn1 and cool_stn2

,heat_stn1,heat_stn2,cool_stn1,cool_stn2
1100,21,20,0,0
1101,21,20,0,0
1102,21,21,0,0
1103,23,23,0,0
1104,0,0,5,9
1105,16,M,0,M
1106,21,20,0,0
1107,5,4,0,0
1108,6,5,0,0
1109,4,4,0,0


In [54]:
weather['heat_stn2'] = weather.apply(lambda x: x['heat_stn1'] if x['heat_stn2'] == 'M' else x['heat_stn2'], axis=1)
weather['cool_stn2'] = weather.apply(lambda x: x['cool_stn1'] if x['cool_stn2'] == 'M' else x['cool_stn2'], axis=1)

In [55]:
weather[weather['avgspeed_stn2']=='M'][['avgspeed_stn2', 'avgspeed_stn1']]

,avgspeed_stn2,avgspeed_stn1
43,M,8.6
872,M,7.3
1033,M,5.8


In [56]:
weather[['avgspeed_stn1', 'avgspeed_stn2']] #the values are similar so i will just replace the missing values with values from avgspeed_stn1

,avgspeed_stn1,avgspeed_stn2
0,9.2,9.6
1,13.4,13.4
2,11.9,13.2
3,10.8,10.4
4,12.0,11.5
...,...,...
1467,12.9,13.6
1468,15.6,14.6
1469,9.9,9.0
1470,5.5,6.5


In [57]:
weather['avgspeed_stn2'] = weather.apply(lambda x: x['avgspeed_stn1'] if x['avgspeed_stn2'] == 'M' else x['avgspeed_stn2'], axis=1)

In [58]:
#imputing values for trace
for x in col_with_trace:
    print(x)
    print(weather[x].unique())
    print()

snowfall_stn1
['0.0' 'T' '0.1']

preciptotal_stn1
['0.00' 'T' '0.13' '0.38' '0.14' '0.07' '0.02' '1.01' '0.04' '0.01' '0.19'
 '0.39' '0.42' '0.28' '0.58' '0.05' '0.16' '0.15' '0.40' '0.30' '0.43'
 '0.11' '1.55' '0.17' '0.03' '1.43' '0.26' '1.31' '0.06' '0.29' '0.41'
 '0.23' '0.09' '1.33' '0.91' '0.37' '2.35' '0.24' '0.34' '0.18' '0.36'
 '0.27' '0.33' '1.28' '0.76' '0.95' '0.08' '0.12' '0.48' '0.52' '0.64'
 '1.24' '0.20' '0.72' '0.73' '1.61' '0.74' '1.05' '2.43' '2.90' '1.23'
 '6.64' '1.44' '0.82' '0.80' '0.32' '0.67' '0.55' '1.73' '0.44' '0.45'
 '3.97' '0.25' '0.68' '0.63' '0.35' '0.93' '0.22' '0.75' '0.89' '0.21'
 '0.46' '1.03' '0.60' '0.81' '0.97' '1.49' '1.02' '0.59' '0.87' '0.85'
 '0.54' '0.79' '2.79' '3.64' '0.10' '0.31' '2.24' '1.06' '1.88' '6.86'
 '0.71' '0.70' '1.11' '0.65' '1.21' '0.77' '1.34' '0.47' '1.12' '0.57'
 '0.51' '1.09' '0.56' '1.35' '0.84' '1.48' '0.92' '0.50' '1.04' '2.09'
 '1.26' '2.60' '2.76' '0.98' '1.58']

preciptotal_stn2
['0.00' 'T' '0.02' '0.60' '0.07' '0.11'

In [59]:
weather['snowfall_stn1'] = weather['snowfall_stn1'].replace('T', 0.01) #since trace value have to be above 0 and lower than 0.1
#since trace value have to be above 0 and below 0.01
weather['preciptotal_stn1'] = weather['preciptotal_stn1'].replace('T', 0.001) 
weather['preciptotal_stn2'] = weather['preciptotal_stn2'].replace('T', 0.001)

for col in col_with_trace:
    weather[col] = pd.to_numeric(weather[col], errors='coerce')

In [60]:
for col in (set(weather.select_dtypes(exclude='number').columns) - set(col_to_convert) - set(col_with_trace) - set(['date'])):
    weather[col] = pd.to_numeric(weather[col], errors='coerce')

In [61]:
weather.isnull().sum()

date                0
tmax_stn1           0
tmin_stn1           0
tavg_stn1           0
depart_stn1         0
dewpoint_stn1       0
wetbulb_stn1        0
heat_stn1           0
cool_stn1           0
sunrise_stn1        0
sunset_stn1         0
depth_stn1          0
snowfall_stn1       0
preciptotal_stn1    0
stnpressure_stn1    0
sealevel_stn1       0
resultspeed_stn1    0
resultdir_stn1      0
avgspeed_stn1       0
tmax_stn2           0
tmin_stn2           0
tavg_stn2           0
dewpoint_stn2       0
wetbulb_stn2        0
heat_stn2           0
cool_stn2           0
preciptotal_stn2    0
stnpressure_stn2    0
sealevel_stn2       0
resultspeed_stn2    0
resultdir_stn2      0
avgspeed_stn2       0
year                0
month               0
day                 0
dtype: int64

In [62]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 0 to 1471
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              1472 non-null   datetime64[ns]
 1   tmax_stn1         1472 non-null   int64         
 2   tmin_stn1         1472 non-null   int64         
 3   tavg_stn1         1472 non-null   int64         
 4   depart_stn1       1472 non-null   int64         
 5   dewpoint_stn1     1472 non-null   int64         
 6   wetbulb_stn1      1472 non-null   int64         
 7   heat_stn1         1472 non-null   int64         
 8   cool_stn1         1472 non-null   int64         
 9   sunrise_stn1      1472 non-null   int64         
 10  sunset_stn1       1472 non-null   int64         
 11  depth_stn1        1472 non-null   int64         
 12  snowfall_stn1     1472 non-null   float64       
 13  preciptotal_stn1  1472 non-null   float64       
 14  stnpressure_stn1  1472 n

In [63]:
train['date'] = pd.to_datetime(train['date'])
train = train.merge(weather, on='date')

In [65]:
weather.to_csv('tableau_dataset/weather.csv', index=False)
spray.to_csv('tableau_dataset/spray.csv', index=False)
train.to_csv('tableau_dataset/train.csv', index=False)